<a href="https://colab.research.google.com/github/AI4Bharat/indicTrans/blob/main/indicTrans_python_interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# clone the repo for running evaluation
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

Cloning into 'indicTrans'...
remote: Enumerating objects: 541, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 541 (delta 181), reused 157 (delta 128), pack-reused 297
Receiving objects: 100% (541/541), 1.53 MiB | 16.32 MiB/s, done.
Resolving deltas: 100% (308/308), done.
/content/indicTrans
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1325, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 1325 (delta 84), reused 89 (delta 41), pack-reused 1178
Receiving objects: 100% (1325/1325), 9.57 MiB | 15.36 MiB/s, done.
Resolving deltas: 100% (688/688), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB

In [12]:
# Install the necessary libraries
!pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
! pip install mosestokenizer subword-nmt
# Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
# !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
!pip install  --use-feature=in-tree-build ./

%cd ..

Cloning into 'fairseq'...
remote: Enumerating objects: 30493, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 30493 (delta 53), reused 105 (delta 49), pack-reused 30367
Receiving objects: 100% (30493/30493), 14.15 MiB | 28.75 MiB/s, done.
Resolving deltas: 100% (22750/22750), done.
/content/indicTrans/fairseq
Processing /content/indicTrans/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for fairseq: filename=fairseq-1.0.0a0+1575f30-cp37-cp37m-linux_x86_64.whl size=9020354 sha256=666bfe9880227061298a86ea75ec3a1b7e27d2df601e6d191ba6657624e94620
  Stored in directory: /tmp/pip-ephem-wheel-cache-bhlm695_/wheels/cf/42/51/9738bb7e2f6f522fa6fdcdaac50d96b3a489f8ad7dc408c27a
Successfully built fairseq
  Attempting uninstall: fairseq
    Found existing installation: fairseq 1.0.0a0+157

/content/indicTrans


In [13]:
# sanity check to see if fairseq is installed
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils

In [4]:
# download the indictrans model


# downloading the indic-en model
!wget https://storage.googleapis.com/samanantar-public/V0.3/models/indic-en.zip
!unzip indic-en.zip

# downloading the en-indic model
# !wget https://storage.googleapis.com/samanantar-public/V0.3/models/en-indic.zip
# !unzip en-indic.zip

# # downloading the indic-indic model
# !wget https://storage.googleapis.com/samanantar-public/V0.3/models/m2m.zip
# !unzip m2m.zip

%cd indicTrans

--2022-01-16 06:39:14--  https://storage.googleapis.com/samanantar-public/V0.3/models/indic-en.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.97.128, 142.251.107.128, 173.194.216.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.97.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4759117228 (4.4G) [application/zip]
Saving to: ‘indic-en.zip’

indic-en.zip        100%[===================>]   4.43G  20.5MB/s    in 83s     

2022-01-16 06:40:37 (54.9 MB/s) - ‘indic-en.zip’ saved [4759117228/4759117228]

Archive:  indic-en.zip
   creating: indic-en/
   creating: indic-en/vocab/
  inflating: indic-en/vocab/bpe_codes.32k.SRC  
  inflating: indic-en/vocab/vocab.SRC  
  inflating: indic-en/vocab/vocab.TGT  
  inflating: indic-en/vocab/bpe_codes.32k.TGT  
   creating: indic-en/final_bin/
  inflating: indic-en/final_bin/preprocess.log  
  inflating: indic-en/final_bin/dict.TGT.txt  
  inflating: indic-en/final_bin

In [5]:
from indicTrans.inference.engine import Model

indic2en_model = Model(expdir='../indic-en')

Initializing vocab and bpe
Initializing model for translation


2022-01-16 06:42:54 | INFO | fairseq.tasks.translation | [SRC] dictionary: 35904 types
2022-01-16 06:42:54 | INFO | fairseq.tasks.translation | [TGT] dictionary: 32088 types


In [11]:
ta_sents = ['அவனுக்கு நம்மைப் தெரியும் என்று தோன்றுகிறது',
            "நாங்கள் உன்னைத் பின்பற்றுவோம் (அ) தொடர்வோம். ",
            'உங்களுக்கு உங்கள் அருகில் இருக்கும் ஒருவருக்கோ இத்தகைய அறிகுறிகள் தென்பட்டால், வீட்டிலேயே இருப்பது, கொரோனா வைரஸ் தொற்று பிறருக்கு வராமல் தடுக்க உதவும்.']


indic2en_model.batch_translate(ta_sents, 'ta', 'en')



100%|██████████| 3/3 [00:00<00:00, 7273.36it/s]
/usr/local/lib/python3.7/dist-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/usr/local/lib/python3.7/dist-packages/fairseq/sequence_generator.py:666: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = bbsz_idx /

['He seems to know us.',
 'We will follow you or continue to follow you',
 'If you develop these symptoms in someone close to you, staying at home can help prevent the spread of Corona virus infection.']

In [7]:

ta_paragraph = """இத்தொற்றுநோய் உலகளாவிய சமூக மற்றும் பொருளாதார சீர்குலைவை ஏற்படுத்தியுள்ளது.இதனால் பெரும் பொருளாதார மந்தநிலைக்குப் பின்னர் உலகளவில் மிகப்பெரிய மந்தநிலை ஏற்பட்டுள்ளது. இது விளையாட்டு,மத, அரசியல் மற்றும் கலாச்சார நிகழ்வுகளை ஒத்திவைக்க அல்லது ரத்து செய்ய வழிவகுத்தது.
அச்சம் காரணமாக முகக்கவசம், கிருமிநாசினி உள்ளிட்ட பொருட்களை அதிக நபர்கள் வாங்கியதால் விநியோகப் பற்றாக்குறை ஏற்பட்டது."""

indic2en_model.translate_paragraph(ta_paragraph, 'ta', 'en')

100%|██████████| 4/4 [00:00<00:00, 4304.06it/s]
/usr/local/lib/python3.7/dist-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/usr/local/lib/python3.7/dist-packages/fairseq/sequence_generator.py:666: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = bbsz_idx /

'The pandemic has caused global social and economic disruption. This has led to the worlds largest recession since the Great Depression. This led to the postponement or cancellation of sporting, religious, political and cultural events. Due to this fear, there was a shortage of supply as most of the people purchased the items like masks, sanitizers etc.'